In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

In [ ]:
import h2o
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='17G')

In [ ]:
import pandas as pd

pd_frame = pd.read_csv('../input/train.csv',nrows=1000000)
pd_frame = pd_frame.drop(['Census_IsFlightingInternal', 'Census_ThresholdOptIn'], axis  = 1)

In [ ]:
pd_frame.shape

In [ ]:
train = h2o.H2OFrame(pd_frame)
#train.head()

In [ ]:
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch

In [ ]:
x = train.columns[1:-1]
y = 'HasDetections'

In [ ]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()

In [ ]:
# Generate a random grid of models and stack them together

# Specify GBM hyperparameters for the grid
hyper_params = {"learn_rate": [0.08,0.09, 0.1, 0.2],
                "max_depth": [3, 4, 5, 6, 9],
                "sample_rate": [0.7, 0.8, 0.9, 1.0],
                "col_sample_rate": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}
search_criteria = {"strategy": "RandomDiscrete", "max_models": 5, "seed": 1}

# Train the grid
grid = H2OGridSearch(model=H2OGradientBoostingEstimator(ntrees=50,
                                                        seed=1,
                                                        nfolds=5,
                                                        fold_assignment="Modulo",
                                                        keep_cross_validation_predictions=True),
                     hyper_params=hyper_params,
                     search_criteria=search_criteria,
                     grid_id="gbm_grid_binomial")
%time grid.train(x=x, y=y, training_frame=train)

In [ ]:
base_gbm_models = grid.model_ids

In [ ]:
# Train a stacked ensemble using the GBM grid
ensemble = H2OStackedEnsembleEstimator(model_id="ensemble_what_else",
                                       base_models=grid.model_ids)

%time ensemble.train(x=x, y=y, training_frame=train)

In [ ]:
test = h2o.import_file("../input/test.csv")

In [ ]:
test = test.drop(['Census_IsFlightingInternal', 'Census_ThresholdOptIn'], axis  = 1)

In [ ]:
test.shape

In [ ]:
# test['Census_IsFlightingInternal'] = test['Census_IsFlightingInternal'].asfactor()
# test['Census_ThresholdOptIn'] = test['Census_ThresholdOptIn'].asfactor()

In [ ]:
preds = ensemble.predict(test)

In [ ]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission.head()

In [ ]:
sample_submission['HasDetections'] = preds['p1'].as_data_frame().values
sample_submission.to_csv('submission.csv', index=False)